In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
#####Inplace class

In [3]:
from collections import namedtuple

Stats = namedtuple('Stats', "plume flower sands goblet circlet")

class Build:
    sub_buffs = ["HP", "ATK", "DEF", "HP%", "ATK%", "DEF%", "EM", "ER", "CRATE", "CDMG"]
    min_subroll = {
                    'HP': 209,
                    'ATK': 14,
                    'DEF': 16,
                    'HP%': 4.1,
                    'ATK%': 4.1,
                    'DEF%': 5.1,
                    'EM': 16,
                    'ER': 4.5,
                    'CRATE': 2.7,
                    'CDMG': 5.4
                  }
    mean_subroll = {
                    'HP': 254.0,
                    'ATK': 16.75,
                    'DEF': 19.75,
                    'HP%': 4.975,
                    'ATK%': 4.975,
                    'DEF%': 6.2,
                    'EM': 19.75,
                    'ER': 5.5,
                    'CRATE': 3.3,
                    'CDMG': 6.6
                  }
    max_subroll = {
                    'HP': 299,
                    'ATK': 19,
                    'DEF': 23,
                    'HP%': 5.8,
                    'ATK%': 5.8,
                    'DEF%': 7.3,
                    'EM': 23,
                    'ER': 6.5,
                    'CRATE': 3.9,
                    'CDMG': 7.8
                  }
    def __init__(self, Weapon, Character, Set, Sands_main, Goblet_main, Circlet_main, constraints,
                 subroll_to_use=None, fit_function=None, max_total_rolls = 9, max_optimizable_subs = 4, locked_subs = {"plume":[], "flower":[], "sands":[], "goblet":[], "circlet":[]}):
        self.max_total_rolls = max_total_rolls
        self.max_optimizable_subs = min(max_optimizable_subs, 4)
        self.locked_subs = locked_subs
        self.fit_function = self.dmg_calc
        if(fit_function is not None):
            self.fit_function = fit_function
        self.subroll = self.mean_subroll
        if(subroll_to_use is not None and subroll_to_use == "MAX"):
            self.subroll = self.max_subroll
        if(subroll_to_use is not None and subroll_to_use == "MIN"):
            self.subroll = self.min_subroll
        self.weapon = Weapon
        self.character = Character
        self.set = Set
        self.mains = Stats("ATK", "HP", Sands_main, Goblet_main, Circlet_main)
        self.subs = Stats([], [], [], [], [])
        self.final_dmg = 0.0
        self.constraints = constraints
        
        self.calculate_stats()
        
    def calculate_stats(self):
        substats = {"HP":0.0, "ATK":0.0, "DEF":0.0, "HP%":0.0, "ATK%":0.0, "DEF%":0.0, "EM":0.0, "ER":0.0, "CRATE":0.0, "CDMG":0.0}
        for (slot,_subs) in self.subs._asdict().items():
            for sub in _subs:
                substats[sub] += self.subroll[sub]
        self.b_atk = self.weapon["BATK"] + self.character["BATK"]
        self.b_def = self.character["BDEF"]
        self.b_hp = self.character["BHP"]
        self.pyro_dmg = self.character["PYRO_DMG"] + self.weapon["PYRO_DMG"] + self.set["PYRO_DMG"]
        self.cryo_dmg = self.character["CRYO_DMG"] + self.weapon["CRYO_DMG"] + self.set["CRYO_DMG"]
        self.hydro_dmg = self.character["HYDRO_DMG"] + self.weapon["HYDRO_DMG"] + self.set["HYDRO_DMG"]
        self.geo_dmg = self.character["GEO_DMG"] + self.weapon["GEO_DMG"] + self.set["GEO_DMG"]
        self.elec_dmg = self.character["ELEC_DMG"] + self.weapon["ELEC_DMG"] + self.set["ELEC_DMG"]
        self.anemo_dmg = self.character["ANEMO_DMG"] + self.weapon["ANEMO_DMG"] + self.set["ANEMO_DMG"]
        self.norm_dmg = self.character["NORM_DMG"] + self.weapon["NORM_DMG"] + self.set["NORM_DMG"]
        self.charge_dmg = self.character["CHARGE_DMG"] + self.weapon["CHARGE_DMG"] + self.set["CHARGE_DMG"]
        self.ele_skill_dmg = self.character["ELE_SKILL_DMG"] + self.weapon["ELE_SKILL_DMG"] + self.set["ELE_SKILL_DMG"]
        self.burst_dmg = self.character["BURST_DMG"] + self.weapon["BURST_DMG"] + self.set["BURST_DMG"]
        self.e_dmg = self.character["ELE_DMG"] + self.weapon["ELE_DMG"] + (46.6 if self.mains.goblet == "ELE_DMG" else 0.0) + self.set["ELE_DMG"]
        self.p_dmg = self.character["PHYS_DMG"] + self.weapon["PHYS_DMG"] + (58.3 if self.mains.goblet == "PHYS_DMG" else 0.0) + self.set["PHYS_DMG"]
        self.r_dmg = self.character["REACT_DMG"] + self.weapon["REACT_DMG"] + self.set["REACT_DMG"]
        self.dmg = self.character["DMG"] + self.weapon["DMG"] + self.set["DMG"]
        self.hp = self.character["HP"] + self.weapon["HP"] + self.set["HP"] + 4780 + substats["HP"]
        self.atk = self.character["ATK"] + self.weapon["ATK"] + self.set["ATK"] + 311 + substats["ATK"]
        self._def = self.character["DEF"] + self.weapon["DEF"] + self.set["DEF"] + substats["DEF"]
        self.p_hp = self.character["HP%"] + self.weapon["HP%"] + self.set["HP%"] + substats["HP%"] + (46.6 if self.mains.sands == "HP%" else 0.0) + (46.6 if self.mains.goblet == "HP%" else 0.0) + (46.6 if self.mains.circlet == "HP%" else 0.0)
        self.p_atk = self.character["ATK%"] + self.weapon["ATK%"] + self.set["ATK%"] + substats["ATK%"] + (46.6 if self.mains.sands == "ATK%" else 0.0) + (46.6 if self.mains.goblet == "ATK%" else 0.0) + (46.6 if self.mains.circlet == "ATK%" else 0.0)
        self.p_def = self.character["DEF%"] + self.weapon["DEF%"] + self.set["DEF%"] + substats["DEF%"] + (46.6 if self.mains.sands == "DEF%" else 0.0) + (46.6 if self.mains.goblet == "DEF%" else 0.0) + (46.6 if self.mains.circlet == "DEF%" else 0.0)
        self.em = self.character["EM"] + self.weapon["EM"] + self.set["EM"] + substats["EM"] + (187 if self.mains.sands == "EM" else 0.0) + (187 if self.mains.goblet == "EM" else 0.0) + (187 if self.mains.circlet == "EM" else 0.0)
        self.er = self.character["ER"] + self.weapon["ER"] + self.set["ER"] + substats["ER"] + (51.8 if self.mains.sands == "ER" else 0.0)
        self.c_rate = self.character["CRATE"] + self.weapon["CRATE"] + self.set["CRATE"] + substats["CRATE"] + (31.1 if self.mains.circlet == "CRATE" else 0.0)
        self.c_dmg = self.character["CDMG"] + self.weapon["CDMG"] + self.set["CDMG"] + substats["CDMG"] + (62.2 if self.mains.circlet == "CDMG" else 0.0)
        self.atk_per_10s = self.character.get("ATK_PER_10s", 10) / (1-self.weapon.get("ASPD%", 0)/100)
        self.f_atk = self.b_atk*(1+self.p_atk/100.0)+self.atk
        self.f_def = self.b_def*(1+self.p_def/100.0)+self._def
        self.f_hp = self.b_hp*(1+self.p_hp/100.0)+self.hp
        self.update_damage()        
    
    def generate_sheet(self):
        return {
            "BATK": self.b_atk,
            "BDEF": self.b_def,
            "BHP": self.b_hp,
            "ELE_DMG": self.e_dmg,
            "PHYS_DMG": self.p_dmg,
            "REACT_DMG": self.r_dmg,
            "DMG": self.dmg,
            "HP": self.hp,
            "ATK": self.atk,
            "DEF": self._def,
            "EM": self.em,
            "ER": self.er,
            "HP%": self.p_hp,
            "ATK%": self.p_atk,
            "DEF%": self.p_def,
            "CRATE": self.c_rate,
            "CDMG": self.c_dmg,
            "ASPD": self.atk_per_10s/10,
            "PYRO_DMG": self.pyro_dmg,
            "CRYO_DMG": self.cryo_dmg,
            "HYDRO_DMG": self.hydro_dmg,
            "GEO_DMG": self.geo_dmg,
            "ELEC_DMG": self.elec_dmg,
            "ANEMO_DMG": self.anemo_dmg,
            "NORM_DMG": self.norm_dmg,
            "CHARGE_DMG": self.charge_dmg,
            "ELE_SKILL_DMG": self.ele_skill_dmg,
            "BURST_DMG": self.burst_dmg,
        }
        
    def is_valid_exchange(self,slot,new=None,old=None):
        max_one_sub_rolls = self.max_total_rolls - 3
        slot_subs = None
        if slot in self.subs._fields:
            slot_subs = getattr(self.subs, slot)
        else:
            return False
        is_on_roll_limit = slot_subs.count(new) == max_one_sub_rolls
        is_main_stat = getattr(self.mains, slot) == new
        is_on_artifact_limit = len(slot_subs) > self.max_total_rolls
        are_we_trying_to_use_unusable_slots = (slot_subs.count(new) > 0 and slot_subs.count(old) == 1)
        is_not_a_locked_sub = old not in self.locked_subs[slot]
        old_is_empty = old == None
        old_exists_in_subs = slot_subs.count(old) == 1
        return not(is_on_roll_limit
                or is_main_stat
                or is_on_artifact_limit
                or are_we_trying_to_use_unusable_slots) and (old_is_empty or old_exists_in_subs) and is_not_a_locked_sub

    def change_substat(self,slot,new=None,old=None):
        max_one_sub_rolls = self.max_total_rolls - 3
        slot_subs = None
        if slot in self.subs._fields:
            slot_subs = getattr(self.subs, slot)
        else:
            return False
        
        if(new is None):
            return False
        
        if(not(self.is_valid_exchange(slot,new,old))):
            return False
            
        if(new is not None and slot_subs.count(new) < max_one_sub_rolls and getattr(self.mains, slot) != new):
            slot_subs.append(new)
            
        if(old is not None and old in slot_subs):
            slot_subs.remove(old)
            
        if(len(slot_subs) > self.max_total_rolls):
            slot_subs.remove(new)
            return False
            
        if(len(slot_subs) >= self.max_optimizable_subs and len(list(set(slot_subs))) < self.max_optimizable_subs):
            slot_subs.append(old)
            slot_subs.remove(new)
            return False
            
        self.calculate_stats()
        return True
    
    def update_damage(self):
        sheet = self.generate_sheet()
        self.final_dmg =  self.fit_function(sheet)
        
    def dmg_calc(self, character_sheet):
        b_atk = character_sheet["BATK"]
        p_atk = character_sheet["ATK%"]
        atk = character_sheet["ATK"]
        dmg = character_sheet["DMG"]
        c_rate = min(character_sheet["CRATE"],100)
        c_dmg = character_sheet["CDMG"]
        return (b_atk*(1.0+p_atk/100.0) + atk) *  (dmg/100.0 + 1.0) * (min(c_rate/100.0,1) * c_dmg/100.0 + 1.0)
    
    def get_possible_subs(self):
        max_one_sub_rolls = self.max_total_rolls - 3
        add_subs = {k:[] for k in self.subs._fields}
        exchange_subs = {k:[] for k in self.subs._fields}
        total_subs = {k:0 for k in self.subs._fields}
        for slot in add_subs:
            main_stat = getattr(self.mains,slot)
            slot_subs = getattr(self.subs, slot)
            locked_subs = self.locked_subs[slot]
            for s in locked_subs:
                if(s not in slot_subs):
                    slot_subs.append(s)
            total_subs[slot] = len(slot_subs)
            if len(slot_subs) >= self.max_optimizable_subs:
                add_subs[slot] = list(set(slot_subs))
                for s in add_subs[slot]:
                    if(slot_subs.count(s) == max_one_sub_rolls):
                        add_subs[slot].remove(s)
            else:
                add_subs[slot] = self.sub_buffs[:]
                if(main_stat in add_subs[slot]):
                    add_subs[slot].remove(main_stat)
                for s in slot_subs:
                    add_subs[slot].remove(s)
                    
            exchange_subs[slot] = self.sub_buffs[:]
            if(main_stat in exchange_subs[slot]):
                    exchange_subs[slot].remove(main_stat)
            for s in list(set(slot_subs)):
                if(exchange_subs[slot].count(s) == max_one_sub_rolls):
                    exchange_subs[slot].remove(s)
                if(s in self.locked_subs[slot]):
                    exchange_subs[slot].remove(s)
        return {"add":add_subs, "exchange": exchange_subs, "total": total_subs}
            
        
    def get_sub_value_rate(self, investment=1):
        sub_add_upgrade = {k:0.0 for k in self.sub_buffs}
        sub_exchange_upgrade = {k:{k_:0.0 for k_ in self.sub_buffs} for k in self.sub_buffs}
        
        sheet = self.generate_sheet()
        
        baseline = self.fit_function(sheet.copy())
        
        added_value = {k:0.0 for k in self.sub_buffs}
        
        for sub in self.sub_buffs:
            s_sheet = sheet.copy()
            s_sheet[sub] += investment*self.subroll[sub]
            added_value[sub] = self.fit_function(s_sheet)
               
        for sub in sub_add_upgrade:
            sub_added_value = 0
            sub_add_upgrade[sub] = added_value[sub] - baseline
        
        for (sub,others) in sub_exchange_upgrade.items():
            for _sub in others:
                ex = sheet.copy()
                ex[sub] += investment*self.subroll[sub]
                ex[_sub] -= investment*self.subroll[_sub]
                sub_exchange_upgrade[sub][_sub] = self.fit_function(ex) - baseline
        
        for (sub,value) in self.constraints.items():
            if value["minimum"] is not None and sheet[sub] < value["minimum"]:
                sub_add_upgrade[sub] = 2*baseline
                for _sub in sub_exchange_upgrade:
                    if(_sub != sub):
                        sub_exchange_upgrade[_sub][sub] = -2*baseline
                        sub_exchange_upgrade[sub][_sub] = 2*baseline
            if(value["minimum"] is not None and  sheet[sub] - self.subroll[sub] < value["minimum"]):
                for _sub in sub_exchange_upgrade:
                    if(_sub != sub):
                        sub_exchange_upgrade[_sub][sub] = -2*baseline
                        sub_exchange_upgrade[sub][_sub] = 0.0
            if value["maximum"] is not None and  sheet[sub] > value["maximum"]:
                sub_add_upgrade[sub] = 2*baseline
                for _sub in sub_exchange_upgrade:
                    if(_sub != sub):
                        sub_exchange_upgrade[_sub][sub] = 2*baseline
                        sub_exchange_upgrade[sub][_sub] = -2*baseline
            if(value["maximum"] is not None and sheet[sub] - self.subroll[sub] > value["maximum"]):
                for _sub in sub_exchange_upgrade:
                    if(_sub != sub):
                        sub_exchange_upgrade[_sub][sub] = 0.0
                        sub_exchange_upgrade[sub][_sub] = -2*baseline
                        
        return {"add": sub_add_upgrade,
                "exchange": sub_exchange_upgrade}
    
    def to_dict(self):
        return {
                 "Character": self.character["Name"]+" "+str(self.character["Lv"]),
                 "Weapon": self.weapon["Name"]+" "+str(self.weapon["Lv"]),
                 "Artifact Set": self.set["Name"],
                 "Sands": self.mains.sands,
                 "Goblet": self.mains.goblet,
                 "Circlet": self.mains.circlet,
                 "Plume_Subs": "_".join([str(self.subs.plume.count(u))+u for u in set(self.subs.plume)]),
                 "Flower_Subs": "_".join([str(self.subs.flower.count(u))+u for u in set(self.subs.flower)]),
                 "Sands_Subs": "_".join([str(self.subs.sands.count(u))+u for u in set(self.subs.sands)]),
                 "Goblet_Subs": "_".join([str(self.subs.goblet.count(u))+u for u in set(self.subs.goblet)]),
                 "Circlet_Subs": "_".join([str(self.subs.circlet.count(u))+u for u in set(self.subs.circlet)]),
                 "Final ATK": self.f_atk,
                 "Final DEF": self.f_def,
                 "Final HP": self.f_hp,
                 "Final EM": self.em,
                 "Final ER": self.er,
                 "Final ELE_DMG Bonus": self.e_dmg + self.pyro_dmg+self.cryo_dmg+self.hydro_dmg+self.geo_dmg+self.elec_dmg+self.anemo_dmg,
                 "Final PHYS_DMG Bonus": self.p_dmg,
                 "Final NORM_DMG": self.norm_dmg,
                 "Final CHARGE_DMG":self.charge_dmg,
                 "Final ELE_SKILL_DMG":self.ele_skill_dmg,
                 "Final BURST_DMG":self.burst_dmg,
                 "Final REACT_DMG": self.r_dmg,
                 "Final ASPD": self.atk_per_10s/10,
                 "Final CRATE": self.c_rate,
                 "Final CDMG": self.c_dmg,
                 "Final DMG": self.final_dmg,
                }

    def optimize_build(self):
        trial = True
        runs = 0
        max_runs = 100
        while(trial and runs < max_runs):
            possible = self.get_possible_subs()
            rates = self.get_sub_value_rate()
            total_operations = 0
            for slot in possible["add"]:
                best_sub = None
                best_removal = None
                p_list = []
                op_type = ""
                valid_exchange = False
                if(possible["total"][slot] < self.max_total_rolls):
                    op_type = "Add"
                    p_list = possible["add"][slot]
                    for sub in p_list:
                        if best_sub is None or rates['add'][best_sub] < rates['add'][sub]:
                            best_sub = sub
                        else:
                            pass
                if(possible["total"][slot] == self.max_total_rolls):
                    op_type = "Exchange"
                    p_list = possible["exchange"][slot]
                    for sub in p_list:
                        for (_sub, value) in rates['exchange'][sub].items():
                            valid_exchange = self.is_valid_exchange(slot,sub,_sub)
                            if (_sub in p_list and valid_exchange and 
                                ((best_sub is None and best_removal == None) or (rates['exchange'][best_sub][best_removal] < rates['exchange'][sub][_sub])) 
                                and rates['exchange'][sub][_sub] > 0):
                                best_sub = sub
                                best_removal = _sub
                            else: 
                                pass
                slot_subs = getattr(self.subs,slot)
                if(best_sub is not None and self.change_substat(slot, best_sub, best_removal)):
                    total_operations += 1
                    rates = self.get_sub_value_rate()
                if(False): 
                    #Log for debug
                    print(slot)
                    print(valid_exchange)
                    print(possible["total"][slot])
                    print(op_type)
                    print(best_sub)
                    print(best_removal)
                    print(p_list)
                    print(getattr(self.subs,slot))
                    print("*******************")
            if total_operations == 0:
                trial = False
            runs+=1

In [4]:
###### General Setup for Weapons/Characters

In [5]:
allStats = ["BATK", "BDEF", "BHP", "PYRO_DMG", "CRYO_DMG", "HYDRO_DMG", "GEO_DMG", "ELEC_DMG", "ANEMO_DMG", 
            "ELE_DMG", "PHYS_DMG", "REACT_DMG", "DMG", "HP", "ATK", "DEF", "HP%", "ATK%", "DEF%", "EM", "ER", 
            "CRATE", "CDMG", "NORM_DMG", "CHARGE_DMG", "ELE_SKILL_DMG", "BURST_DMG"]

artifact_set_2pc = {
    "BC": {"PHYS_DMG":25},
    "GF": {"ATK%":18},
    "WT": {"EM":80},
    "PF": {"PHYS_DMG":25},
    #"TF": {"ELEC_DMG":15},#, "DT": "Electro" },
    #"VV": {"ANEMO_DMG":15},#, "DT": "Anemo"  },
    #"AP": {"GEO_DMG":15},#, "DT": "Geo"},
    "CW": {"PYRO_DMG":15},#, "DT": "Pyro"},
    #"BS": {"CRYO_DMG":15},#, "DT": "Cryo"},
    #"HD": {"HYDRO_DMG":15},#, "DT": "Hydro", },
    "NO": {"BURST_DMG":20},#, "T": "Burst"},
    #"TM": {"HP%": 20}
}
artifact_set_4pc = {
    "BC": {"PHYS_DMG":25},
    "GF": {"ATK%":18},#, "AT": "Normal", "WT": ["Claymore", "Polearm", "Sword},
    "GF!": {"ATK%":18, "NOMR_DMG": 35, "CHARGE_DMG": 35},#, "AT": "Normal", "WT": ["Claymore", "Polearm", "Sword},
    "WT!": {"EM":80, "CHARGE_DMG": 35}, #"AT": "Charge", "WT": ["Bow", "Catalyst},
    "WT": {"EM":80}, #"AT": "Charge", "WT": ["Bow", "Catalyst},
    "RT!": {"NORM_DMG": 40, "CHARGE_DMG": 40},#, "AT": ["Charge","Normal},
    "TS": {"DMG": 35},
    "LW": {"DMG": 35},
    "PF": {"PHYS_DMG":25},
    "PF!": {"PHYS_DMG":50, "ATK%": 18},
    #"TF": {"ELEC_DMG":15, "REACT_DMG":40},#, "DT": "Electro" },
    #"VV": {"ANEMO_DMG":15, "REACT_DMG":60},#, "DT": "Anemo"  },
    #"AP": {"GEO_DMG":15},#, "DT": "Geo"},
    #"AP!": {"GEO_DMG":15, "ELE_DMG":35},#, "DT": "Geo"}
    "CW": {"PYRO_DMG":15, "REACT_DMG":40},#, "DT": "Pyro"},
    "CW!": {"PYRO_DMG":37.5, "REACT_DMG":40},#, "DT": "Pyro"},
    #"BS": {"CRYO_DMG":15, "CRATE": 20},#, "DT": "Cryo"},
    #"BS!": {"CRYO_DMG":15, "CRATE": 40},#, "DT": "Cryo"},
    #"HD": {"HYDRO_DMG":15},#, "DT": "Hydro", },
    "NO": {"BURST_DMG":20},#, "T": "Burst"},
    "NO!": {"BURST_DMG":20, "ATK%": 20},#, "T": "Burst"},
    #"TM": {"HP%": 20},
    #"TM!": {"HP%": 20, "ATK%": 20}    
}

sets_4pc = {k:v for (k,v) in artifact_set_4pc.items()}

for (i,v1) in artifact_set_2pc.items():
    for (j,v2) in artifact_set_2pc.items():
        if (i!=j):
            sets_4pc[i+j] = {**v1, **v2}


for (k,v) in sets_4pc.items():
    sec_stats = {k:0.0 for k in allStats}
    sets_4pc[k] = {**sec_stats,**v}
    
sets_4pc = {name:{"Name":name, **_set} for (name,_set) in sets_4pc.items()}

artifact_slots = {
    #"HEAD": ["CRATE", "CDMG", "EM", "ATK%", "HP%", "DEF%"],
    #"SANDS": ["ER", "EM", "ATK%", "HP%", "DEF%"],
    #"GOBLET": ["ELE_DMG", "PHYS_DMG", "ATK%", "HP%", "DEF%", "EM"],
    "HEAD": ["CRATE", "CDMG", "ATK%"],
    "SANDS": ["ER", "ATK%"],
    "GOBLET": ["ELE_DMG", "PHYS_DMG", "ATK%"],
    "FLOWER": ["HP"],
    "PLUME": ["ATK"],
}

weapons = {
    "SKY": {"BATK": 674, "ER%": 36.8, "DMG":8},
    "WGS": {"BATK": 608, "ATK%": 69.6},
    "WGS!": {"BATK": 608, "ATK%": 109.6},
    "UNF(0)": {"BATK": 608, "ATK%": 49.6},
    "UNF(1)": {"BATK": 608, "ATK%": 53.6},
    "UNF(2)": {"BATK": 608, "ATK%": 57.6},
    "UNF(3)": {"BATK": 608, "ATK%": 61.6},
    "UNF(4)": {"BATK": 608, "ATK%": 65.6},
    "UNF(5)": {"BATK": 608, "ATK%": 69.6},
    "UNF(1)!": {"BATK": 608, "ATK%": 57.6},
    "UNF(2)!": {"BATK": 608, "ATK%": 65.6},
    "UNF(3)!": {"BATK": 608, "ATK%": 73.6},
    "UNF(4)!": {"BATK": 608, "ATK%": 81.6},
    "UNF(5)!": {"BATK": 608, "ATK%": 89.6},
    "SBP":{"BATK": 741, "PHYS_DMG": 20.7, "ATK%": 16},
    "SBP!":{"BATK": 741, "PHYS_DMG": 20.7, "ATK%": 36, "ASPD%": 12},
    "ARC":{"BATK": 565, "ATK%": 27.6},
    "BLK(0)":{"BATK": 510, "CMDG": 55.1},
    "BLK(1)":{"BATK": 510, "CMDG": 55.1, "ATK%": 12},
    "BLK(2)":{"BATK": 510, "CMDG": 55.1, "ATK%": 24},
    "BLK(3)":{"BATK": 510, "CMDG": 55.1, "ATK%": 36},
    "BLK R5(0)":{"BATK": 510, "CMDG": 55.1},
    "BLK R5(1)":{"BATK": 510, "CMDG": 55.1, "ATK%": 24},
    "BLK R5(2)":{"BATK": 510, "CMDG": 55.1, "ATK%": 48},
    "BLK R5(3)":{"BATK": 510, "CMDG": 55.1, "ATK%": 72},
    "RYL(0)":{"BATK": 565, "CRATE": 0, "ATK%": 27.6},
    "RYL(1)":{"BATK": 565, "CRATE": 8, "ATK%": 27.6},
    "RYL(2)":{"BATK": 565, "CRATE": 16, "ATK%": 27.6},
    "RYL(3)":{"BATK": 565, "CRATE": 24, "ATK%": 27.6},
    "RYL(4)":{"BATK": 565, "CRATE": 32, "ATK%": 27.6},
    "RYL(5)":{"BATK": 565, "CRATE": 40, "ATK%": 27.6},
}

weapons = {name:{"Name":name, "Lv":90, **weapon} for (name,weapon) in weapons.items()}
    
characters = {"Diluc": {
    "Name": "Diluc",
    "Lv": 80,
    "BHP": 12068,
    "BATK": 311,
    "BDEF": 729,
    "CRATE": 24.2,
    "CDMG": 50,
    "EM": 0,
    "PYRO_DMG": 0,
    "CRYO_DMG": 0,
    "HYDRO_DMG": 0,
    "GEO_DMG": 0,
    "ELEC_DMG": 0,
    "ANEMO_DMG": 0,
    "ELE_DMG": 0,
    "PHYS_DMG": 0, 
    "REACT_DMG": 0,
    "ER": 100,
    "ATK_PER_10s": 14,
    #"DT": "Pyro",
    #"WT": "Claymore",
    #"Talents": {
    #    "Normal":[
    #        {"ATK":153.32, "HP": 0, "DEF": 0},
    #        {"ATK":149.79, "HP": 0, "DEF": 0},
    #        {"ATK":168.9, "}HP": 0, "DEF": 0},
    #        {"ATK":229.03, "HP": 0, "DEF": 0}
    #    ], 
    #    "Charge":[], 
    #    "Elemental":[
    #        {"ATK":151.04, "HP": 0, "DEF": 0},
    #        {"ATK":156.16, "HP": 0, "DEF": 0},
    #        {"ATK":206.08, "HP": 0, "DEF": 0}
    #    ], 
    #    "Burst":[
    #        {"ATK":326.4, "HP": 0, "DEF": 0},
    #        {"ATK":96, "HP": 0, "DEF": 0},
    #        {"ATK":326.4, "HP": 0, "DEF": 0}
    #    ]
    #},
    #"Rotation": "NNNENNNENNNENNNQNNNENNNENNNE"
}}

constraints = {
    "EM": {"minimum":None, "maximum": None},
    "ER": {"minimum":100, "maximum": None}
}

allStats = {k:0.0 for k in allStats}
Base_Multiplier = {
    "Burning":0.25,
    "Superconduct":0.5,
    "Swirl":0.6,
    "Electro-Charged":1.2,
    "Shattered":1.5,
    "Overloaded":2
}

In [6]:
def my_fit_function(sheet):
    b_atk = sheet["BATK"]
    p_atk = sheet["ATK%"]
    atk = sheet["ATK"]
    edmg = sheet["DMG"] + sheet["ELE_DMG"] + sheet["PYRO_DMG"]
    pdmg = sheet["DMG"] + sheet["PHYS_DMG"]
    normal_dmg = sheet["NORM_DMG"]
    skill_dmg = sheet["ELE_SKILL_DMG"]
    burst_dmg = sheet["BURST_DMG"]
    c_rate = min(sheet["CRATE"],100)
    c_dmg = sheet["CDMG"]
    em = sheet["EM"]
    react_dmg = sheet["REACT_DMG"]
    # 2.78 for melt/vap or 6.67 for everything else
    reaction_type_multiplier = 2.78
    # 1.5 for reverse vap, reverse melt; 2 for vap/melt
    # For everything else: Base Multiplier × Character Level Multiplier
    # Base Multiplier up there. Level Multiplier: 80 -> 946.4, 90 -> 1202.8, 100 -> 1674.8
    react_multiplier = 1.5
    # for vap/melt
    react_final_multiplier = react_multiplier * (1+(em/(1400+em))*reaction_type_multiplier + react_dmg/100.0)
    #QAAAEAAAEAAAEAAA (vap on Q and E)
    #return 1.51*(b_atk*(1.0+p_atk/100.0) + atk) *  (edmg/100.0 + 1.0) * (min(c_rate/100.0,1) * c_dmg/100.0 + 1.0)
    return (((326 + 96*3)/100.0*(1+burst_dmg/100.0) * react_final_multiplier + 4*472/100.0*(1+normal_dmg/100.0) + (151 + 156 + 206)/100.0*(1+skill_dmg/100.0)* react_final_multiplier)
            *(b_atk*(1.0+p_atk/100.0) + atk) *  (edmg/100.0 + 1.0) * (min(c_rate/100.0,1) * c_dmg/100.0 + 1.0))/(4+3*4)

In [7]:
##### Generate All builds and optmize them
builds = {}
start_time = time.time()
for (char_name,char_data) in characters.items():
    char_data = {**allStats, **char_data} 
    for (weapon_name,weapon_data) in weapons.items():
        weapon_data = {**allStats, **weapon_data}
        for (set_name,set_data) in sets_4pc.items():
            set_data = {**allStats, **set_data}
            for sands in artifact_slots['SANDS']:
                for goblet in artifact_slots['GOBLET']:
                    for circlet in artifact_slots['HEAD']:
                        build = Build(Weapon=weapon_data, 
                                            Character=char_data, 
                                            Set=set_data, 
                                            Sands_main=sands, 
                                            Goblet_main=goblet, 
                                            Circlet_main=circlet, 
                                            constraints=constraints,
                                             fit_function=my_fit_function)
                        build.optimize_build()
                        build_id = char_name+"_"+weapon_name+"_"+set_name+"_"+sands+"_"+goblet+"_"+circlet
                        builds[build_id] = build.to_dict()
print("Total Time --- %s seconds ---" % (time.time() - start_time))

Total Time --- 1006.7106423377991 seconds ---


In [8]:
builds_df = pd.DataFrame(builds).transpose()
builds_df

,Character,Weapon,Artifact Set,Sands,Goblet,Circlet,Plume_Subs,Flower_Subs,Sands_Subs,Goblet_Subs,...,Final PHYS_DMG Bonus,Final NORM_DMG,Final CHARGE_DMG,Final ELE_SKILL_DMG,Final BURST_DMG,Final REACT_DMG,Final ASPD,Final CRATE,Final CDMG,Final DMG
Diluc_SKY_BC_ER_ELE_DMG_CRATE,Diluc 80,SKY 90,BC,ER,ELE_DMG,CRATE,3CDMG_3ATK%_1EM_2CRATE,3CDMG_4ATK%_1EM_1CRATE,3CDMG_3ATK%_1EM_2CRATE,4CDMG_3ATK%_1EM_1CRATE,...,25.0,0.0,0.0,0.0,0.0,0.0,1.4,75.1,155.6,17482.153465
Diluc_SKY_BC_ER_ELE_DMG_CDMG,Diluc 80,SKY 90,BC,ER,ELE_DMG,CDMG,2CDMG_3ATK%_1EM_3CRATE,1CDMG_4ATK%_1EM_3CRATE,2CDMG_3ATK%_1EM_3CRATE,1CDMG_3ATK%_1EM_4CRATE,...,25.0,0.0,0.0,0.0,0.0,0.0,1.4,77.0,151.8,17484.604209
Diluc_SKY_BC_ER_ELE_DMG_ATK%,Diluc 80,SKY 90,BC,ER,ELE_DMG,ATK%,3CDMG_3ATK%_1EM_2CRATE,2CDMG_3ATK%_1EM_3CRATE,3CDMG_3ATK%_1EM_2CRATE,3CDMG_2ATK%_1EM_3CRATE,...,25.0,0.0,0.0,0.0,0.0,0.0,1.4,70.4,142.4,17384.672507
Diluc_SKY_BC_ER_PHYS_DMG_CRATE,Diluc 80,SKY 90,BC,ER,PHYS_DMG,CRATE,3CDMG_3ATK%_1EM_2CRATE,3CDMG_4ATK%_1EM_1CRATE,3CDMG_3ATK%_1EM_2CRATE,4CDMG_3ATK%_1EM_1CRATE,...,83.3,0.0,0.0,0.0,0.0,0.0,1.4,75.1,155.6,12212.629847
Diluc_SKY_BC_ER_PHYS_DMG_CDMG,Diluc 80,SKY 90,BC,ER,PHYS_DMG,CDMG,2CDMG_3ATK%_1EM_3CRATE,1CDMG_4ATK%_1EM_3CRATE,2CDMG_3ATK%_1EM_3CRATE,1CDMG_3ATK%_1EM_4CRATE,...,83.3,0.0,0.0,0.0,0.0,0.0,1.4,77.0,151.8,12214.34188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Diluc_RYL(5)_NOCW_ATK%_PHYS_DMG_CDMG,Diluc 80,RYL(5) 90,NOCW,ATK%,PHYS_DMG,CDMG,6CDMG_1ATK%_1EM_1CRATE,5CDMG_1ATK%_1EM_2CRATE,1ATK_4CDMG_1EM_3CRATE,3CDMG_3ATK%_1EM_2CRATE,...,58.3,0.0,0.0,0.0,20.0,0.0,1.4,100.5,231.0,22103.704145
Diluc_RYL(5)_NOCW_ATK%_PHYS_DMG_ATK%,Diluc 80,RYL(5) 90,NOCW,ATK%,PHYS_DMG,ATK%,5CDMG_1ATK%_1EM_2CRATE,5CDMG_1ATK%_1EM_2CRATE,1ATK_5CDMG_1EM_2CRATE,5CDMG_1ATK%_1EM_2CRATE,...,58.3,0.0,0.0,0.0,20.0,0.0,1.4,97.2,215.0,21971.237831
Diluc_RYL(5)_NOCW_ATK%_ATK%_CRATE,Diluc 80,RYL(5) 90,NOCW,ATK%,ATK%,CRATE,6CDMG_1ATK%_1EM_1CRATE,1ATK_6CDMG_1ATK%_1EM,1ATK_6CDMG_1EM_1CRATE,1ATK_6CDMG_1EM_1HP,...,0.0,0.0,0.0,0.0,20.0,0.0,1.4,101.9,248.0,25089.965131
Diluc_RYL(5)_NOCW_ATK%_ATK%_CDMG,Diluc 80,RYL(5) 90,NOCW,ATK%,ATK%,CDMG,6CDMG_1ATK%_1EM_1CRATE,5CDMG_1ATK%_1EM_2CRATE,1ATK_5CDMG_1EM_2CRATE,1ATK_5CDMG_1EM_2CRATE,...,0.0,0.0,0.0,0.0,20.0,0.0,1.4,100.5,250.8,26020.812891


In [9]:
builds_df["Percentage of Best DMG"] = builds_df["Final DMG"]*100.0/builds_df["Final DMG"].max()

In [10]:
comp_df = builds_df.query("`Artifact Set` == 'GFWT'")

In [11]:
comp_df.sort_values("Final DMG", ascending=False).drop_duplicates(['Weapon']).filter(["Weapon", "Sands", "Goblet", "Circlet", "Final ATK", "Final CRATE", "Final CDMG", "Final DMG", "Percentage of Best DMG"])

,Weapon,Sands,Goblet,Circlet,Final ATK,Final CRATE,Final CDMG,Final DMG,Percentage of Best DMG
Diluc_WGS!_GFWT_ATK%_ELE_DMG_CDMG,WGS! 90,ATK%,ELE_DMG,CDMG,3047.279,96.8,191.4,32741.258225,75.830879
Diluc_UNF(5)!_GFWT_ATK%_ELE_DMG_CDMG,UNF(5)! 90,ATK%,ELE_DMG,CDMG,2863.479,96.8,191.4,30766.433058,71.257056
Diluc_RYL(5)_GFWT_ATK%_ELE_DMG_CDMG,RYL(5) 90,ATK%,ELE_DMG,CDMG,2333.239,100.5,244.2,30247.48157,70.05513
Diluc_UNF(4)!_GFWT_ATK%_ELE_DMG_CDMG,UNF(4)! 90,ATK%,ELE_DMG,CDMG,2789.959,96.8,191.4,29976.502991,69.427526
Diluc_UNF(3)!_GFWT_ATK%_ELE_DMG_CDMG,UNF(3)! 90,ATK%,ELE_DMG,CDMG,2716.439,96.8,191.4,29186.572924,67.597997
Diluc_RYL(4)_GFWT_ATK%_ELE_DMG_CDMG,RYL(4) 90,ATK%,ELE_DMG,CDMG,2289.658,99.1,237.6,28928.942588,67.001309
Diluc_UNF(5)_GFWT_ATK%_ELE_DMG_CDMG,UNF(5) 90,ATK%,ELE_DMG,CDMG,2679.679,96.8,191.4,28791.60789,66.683233
Diluc_WGS_GFWT_ATK%_ELE_DMG_CDMG,WGS 90,ATK%,ELE_DMG,CDMG,2679.679,96.8,191.4,28791.60789,66.683233
Diluc_SBP!_GFWT_ATK%_ELE_DMG_CDMG,SBP! 90,ATK%,ELE_DMG,CDMG,2664.16,96.8,191.4,28624.865171,66.297046
Diluc_UNF(2)!_GFWT_ATK%_ELE_DMG_CDMG,UNF(2)! 90,ATK%,ELE_DMG,CDMG,2642.919,96.8,191.4,28396.642857,65.768468


In [12]:
builds_df.sort_values("Final DMG", ascending=False).drop_duplicates(['Weapon'])

,Character,Weapon,Artifact Set,Sands,Goblet,Circlet,Plume_Subs,Flower_Subs,Sands_Subs,Goblet_Subs,...,Final NORM_DMG,Final CHARGE_DMG,Final ELE_SKILL_DMG,Final BURST_DMG,Final REACT_DMG,Final ASPD,Final CRATE,Final CDMG,Final DMG,Percentage of Best DMG
Diluc_WGS!_CW!_ATK%_ELE_DMG_CDMG,Diluc 80,WGS! 90,CW!,ATK%,ELE_DMG,CDMG,4CDMG_1ATK%_1EM_3CRATE,3CDMG_1ATK%_1EM_4CRATE,1ATK_2CDMG_1EM_5CRATE,3CDMG_1ATK%_1EM_4CRATE,...,0.0,0.0,0.0,0.0,40.0,1.4,96.8,191.4,43176.682947,100.0
Diluc_UNF(5)!_CW!_ATK%_ELE_DMG_CDMG,Diluc 80,UNF(5)! 90,CW!,ATK%,ELE_DMG,CDMG,4CDMG_1ATK%_1EM_3CRATE,3CDMG_1ATK%_1EM_4CRATE,1ATK_2CDMG_1EM_5CRATE,3CDMG_1ATK%_1EM_4CRATE,...,0.0,0.0,0.0,0.0,40.0,1.4,96.8,191.4,40422.948526,93.622172
Diluc_RYL(5)_CW!_ATK%_ELE_DMG_CDMG,Diluc 80,RYL(5) 90,CW!,ATK%,ELE_DMG,CDMG,6CDMG_1ATK%_1EM_1CRATE,5CDMG_1ATK%_1EM_2CRATE,1ATK_4CDMG_1EM_3CRATE,3CDMG_3ATK%_1EM_2CRATE,...,0.0,0.0,0.0,0.0,40.0,1.4,100.5,231.0,39334.309342,91.100813
Diluc_UNF(4)!_CW!_ATK%_ELE_DMG_CDMG,Diluc 80,UNF(4)! 90,CW!,ATK%,ELE_DMG,CDMG,4CDMG_1ATK%_1EM_3CRATE,3CDMG_1ATK%_1EM_4CRATE,1ATK_2CDMG_1EM_5CRATE,3CDMG_1ATK%_1EM_4CRATE,...,0.0,0.0,0.0,0.0,40.0,1.4,96.8,191.4,39321.454757,91.071041
Diluc_UNF(3)!_CW!_ATK%_ELE_DMG_CDMG,Diluc 80,UNF(3)! 90,CW!,ATK%,ELE_DMG,CDMG,4CDMG_1ATK%_1EM_3CRATE,3CDMG_1ATK%_1EM_4CRATE,1ATK_2CDMG_1EM_5CRATE,3CDMG_1ATK%_1EM_4CRATE,...,0.0,0.0,0.0,0.0,40.0,1.4,96.8,191.4,38219.960989,88.51991
Diluc_WGS_CW!_ATK%_ELE_DMG_CDMG,Diluc 80,WGS 90,CW!,ATK%,ELE_DMG,CDMG,4CDMG_1ATK%_1EM_3CRATE,3CDMG_1ATK%_1EM_4CRATE,1ATK_2CDMG_1EM_5CRATE,3CDMG_1ATK%_1EM_4CRATE,...,0.0,0.0,0.0,0.0,40.0,1.4,96.8,191.4,37669.214105,87.244345
Diluc_UNF(5)_CW!_ATK%_ELE_DMG_CDMG,Diluc 80,UNF(5) 90,CW!,ATK%,ELE_DMG,CDMG,4CDMG_1ATK%_1EM_3CRATE,3CDMG_1ATK%_1EM_4CRATE,1ATK_2CDMG_1EM_5CRATE,3CDMG_1ATK%_1EM_4CRATE,...,0.0,0.0,0.0,0.0,40.0,1.4,96.8,191.4,37669.214105,87.244345
Diluc_RYL(4)_CW!_ATK%_ELE_DMG_CDMG,Diluc 80,RYL(4) 90,CW!,ATK%,ELE_DMG,CDMG,6CDMG_1ATK%_1EM_1CRATE,3CDMG_2ATK%_1EM_3CRATE,1ATK_4CDMG_1EM_3CRATE,4CDMG_2ATK%_1EM_2CRATE,...,0.0,0.0,0.0,0.0,40.0,1.4,99.1,224.4,37572.133703,87.0195
Diluc_UNF(4)_CW!_ATK%_ELE_DMG_CDMG,Diluc 80,UNF(4) 90,CW!,ATK%,ELE_DMG,CDMG,4CDMG_1ATK%_1EM_3CRATE,3CDMG_1ATK%_1EM_4CRATE,1ATK_2CDMG_1EM_5CRATE,3CDMG_1ATK%_1EM_4CRATE,...,0.0,0.0,0.0,0.0,40.0,1.4,96.8,191.4,37118.46722,85.968779
Diluc_UNF(2)!_CW!_ATK%_ELE_DMG_CDMG,Diluc 80,UNF(2)! 90,CW!,ATK%,ELE_DMG,CDMG,4CDMG_1ATK%_1EM_3CRATE,3CDMG_1ATK%_1EM_4CRATE,1ATK_2CDMG_1EM_5CRATE,3CDMG_1ATK%_1EM_4CRATE,...,0.0,0.0,0.0,0.0,40.0,1.4,96.8,191.4,37118.46722,85.968779
